##### Load modules

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

##### Load and preprocess building meta file

In [3]:
df_building = reduce_mem_usage(pd.read_csv('building_metadata.csv'))
df_building['primary_use'] = LabelEncoder().fit_transform(df_building.primary_use)

Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.8%


##### Preprocess train dataset

In [4]:
# load csv files
df_meter = reduce_mem_usage(pd.read_csv('train.csv'))
df_weather = reduce_mem_usage(pd.read_csv('weather_train.csv'))

# create a dense timestamp (bcz weather's is sparse)
timestamp = df_meter.timestamp.unique()
timestamp = np.sort(timestamp)
timestamp = pd.DataFrame(data=timestamp, columns=['timestamp'])

dt = pd.DatetimeIndex(timestamp.timestamp)
timestamp['day'] = dt.day
timestamp['hour'] = dt.hour
timestamp['weekday'] = dt.weekday

# project each site's weather data to the dense timestamp and do interpolation
dfs = []
for idx, group in df_weather.groupby('site_id'):
    group = pd.merge(timestamp, group, on='timestamp', how='left')
    group = group.interpolate(limit_direction='both')
#     group['air_temperature_12'] = group.air_temperature.rolling(12).mean().fillna(method='bfill')
    group['air_temperature_24'] = group.air_temperature.rolling(24).mean().fillna(method='bfill')
    group['air_temperature_48'] = group.air_temperature.rolling(48).mean().fillna(method='bfill')
    group['air_temperature_96'] = group.air_temperature.rolling(96).mean().fillna(method='bfill')
#     group['dew_temperature_12'] = group.dew_temperature.rolling(12).mean().fillna(method='bfill')
    group['dew_temperature_24'] = group.dew_temperature.rolling(24).mean().fillna(method='bfill')
    group['dew_temperature_48'] = group.dew_temperature.rolling(48).mean().fillna(method='bfill')
    group['dew_temperature_96'] = group.dew_temperature.rolling(96).mean().fillna(method='bfill')
#     group['sea_level_pressure_12'] = group.sea_level_pressure.rolling(12).mean().fillna(method='bfill')
    group['sea_level_pressure_24'] = group.sea_level_pressure.rolling(24).mean().fillna(method='bfill')
    group['sea_level_pressure_48'] = group.sea_level_pressure.rolling(48).mean().fillna(method='bfill')
    group['sea_level_pressure_96'] = group.sea_level_pressure.rolling(96).mean().fillna(method='bfill')
    dfs.append(group)
df_weather = pd.concat(dfs)

# merge meter data with the building data
df = pd.merge(df_meter, df_building, on='building_id', how='left')

# merge meter data with the weather data
df = pd.merge(df, df_weather, on=['site_id', 'timestamp'], how='left')

# mathmatical conversion
df['meter_reading'] = np.log1p(df.meter_reading)
df['wind_direction'] = np.round(df.wind_direction, -1)

# reduce memory size
where = df.meter == 0
where &= df.site_id == 0
where &= df.timestamp < '2016-05-20'
df.drop(index=df[where].index, inplace=True)
df.drop(columns='timestamp', inplace=True)
df = reduce_mem_usage(df)

# save to the disk
df.to_pickle('train.pickle')

# Collect memory
del df, dfs, idx, group, df_meter, df_weather, timestamp, dt

Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 173.90 MB
Decreased by 71.8%
Memory usage of dataframe is 9.60 MB
Memory usage after optimization is: 2.65 MB
Decreased by 72.4%
Memory usage of dataframe is 2615.03 MB
Memory usage after optimization is: 1099.07 MB
Decreased by 58.0%


##### Preprocess test dataset

In [5]:
## load csv files
df_meter = reduce_mem_usage(pd.read_csv('test.csv'))
df_weather = reduce_mem_usage(pd.read_csv('weather_test.csv'))

# create a dense timestamp (bcz weather's is sparse)
timestamp = df_meter.timestamp.unique()
timestamp = np.sort(timestamp)
timestamp = pd.DataFrame(data=timestamp, columns=['timestamp'])

dt = pd.DatetimeIndex(timestamp.timestamp)
timestamp['day'] = dt.day
timestamp['hour'] = dt.hour
timestamp['weekday'] = dt.weekday

# project each site's weather data to the dense timestamp and do interpolation
dfs = []
for idx, group in df_weather.groupby('site_id'):
    group = pd.merge(timestamp, group, on='timestamp', how='left')
    group = group.interpolate(limit_direction='both')
#     group['air_temperature_12'] = group.air_temperature.rolling(12).mean().fillna(method='bfill')
    group['air_temperature_24'] = group.air_temperature.rolling(24).mean().fillna(method='bfill')
    group['air_temperature_48'] = group.air_temperature.rolling(48).mean().fillna(method='bfill')
    group['air_temperature_96'] = group.air_temperature.rolling(96).mean().fillna(method='bfill')
#     group['dew_temperature_12'] = group.dew_temperature.rolling(12).mean().fillna(method='bfill')
    group['dew_temperature_24'] = group.dew_temperature.rolling(24).mean().fillna(method='bfill')
    group['dew_temperature_48'] = group.dew_temperature.rolling(48).mean().fillna(method='bfill')
    group['dew_temperature_96'] = group.dew_temperature.rolling(96).mean().fillna(method='bfill')
#     group['sea_level_pressure_12'] = group.sea_level_pressure.rolling(12).mean().fillna(method='bfill')
    group['sea_level_pressure_24'] = group.sea_level_pressure.rolling(24).mean().fillna(method='bfill')
    group['sea_level_pressure_48'] = group.sea_level_pressure.rolling(48).mean().fillna(method='bfill')
    group['sea_level_pressure_96'] = group.sea_level_pressure.rolling(96).mean().fillna(method='bfill')
    dfs.append(group)
df_weather = pd.concat(dfs)

# merge meter data with the building data
df = pd.merge(df_meter, df_building, on='building_id', how='left')

# merge meter data with the weather data
df = pd.merge(df, df_weather, on=['site_id', 'timestamp'], how='left')

# mathmatical conversion
df['wind_direction'] = np.round(df.wind_direction, -1)

# reduce memory size
df.drop(columns='timestamp', inplace=True)
df = reduce_mem_usage(df)

# save to the disk
df.to_pickle('test.pickle')

# Collect memory
del df, dfs, idx, group, df_meter, df_weather, timestamp, dt

Memory usage of dataframe is 1272.51 MB
Memory usage after optimization is: 358.65 MB
Decreased by 71.8%
Memory usage of dataframe is 19.04 MB
Memory usage after optimization is: 5.25 MB
Decreased by 72.4%
Memory usage of dataframe is 5487.70 MB
Memory usage after optimization is: 2385.96 MB
Decreased by 56.5%
